In [1]:
import os
import chromadb
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv

# Load environment variables (ensure OPENAI_API_KEY is set)
load_dotenv()


True

In [2]:

# ✅ Initialize ChromaDB client
chroma_client = chromadb.PersistentClient(path="../chroma_db")
collection = chroma_client.get_collection(name="rfp_proposals")


In [ ]:
collection

Collection(name=rfp_proposals)

: 

In [3]:

# ✅ Initialize LLM and Embeddings
embedding_function = OpenAIEmbeddings()
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)


In [4]:

# ✅ Retrieve all unique vendors from metadata
def get_unique_suppliers():
    results = collection.get(include=["metadatas"])
    suppliers = {metadata["supplier"] for metadata in results["metadatas"] if "supplier" in metadata}
    return list(suppliers)


In [5]:
# ✅ Retrieve all proposal chunks for a given supplier using metadata filtering
def retrieve_chunks_for_supplier(supplier_name):
    results = collection.get(
        where={"supplier": supplier_name}, 
        include=["documents"]
    )
    return results["documents"] if results and "documents" in results else []


In [6]:
# ✅ Generate structured output using LLM
def generate_structured_output(supplier_name, documents):
    context = "\n".join(documents)
    
    prompt_template = PromptTemplate(
        input_variables=["supplier", "context"],
        template="""
        Given the following supplier proposal documents for {supplier}, extract and structure the key details:
        
        - **Pricing**
        - **Contract Terms**
        - **Compliance**
        - **Technical Capabilities**
        - **Support & SLAs**
        - **Implementation Timeline**
        - **Past Performance**
        - **Customization & Flexibility**
        
        Provide a clear and structured output in markdown format.
        
        Context:
        {context}
        """
    )
    chain = LLMChain(llm=llm, prompt=prompt_template)
    return chain.run(supplier=supplier_name, context=context)


In [7]:

# ✅ Main function to process all vendors
def process_vendors():
    suppliers = get_unique_suppliers()
    final_output = ""  # Store structured outputs
    
    for supplier in suppliers:
        print(f"Processing {supplier}...")
        documents = retrieve_chunks_for_supplier(supplier)
        if not documents:
            print(f"⚠️ No data found for {supplier}, skipping.")
            continue
        structured_data = generate_structured_output(supplier, documents)
        final_output += f"\n## {supplier}\n" + structured_data + "\n\n"
    
    print("✅ Finished processing all vendors.")
    return final_output


In [8]:

if __name__ == "__main__":
    structured_vendor_comparison = process_vendors()
    print(structured_vendor_comparison)


Processing CloudNexus Technologies 
Headquarters...


C:\Users\neural_ninja\AppData\Local\Temp\ipykernel_10712\647575809.py:25: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt_template)
C:\Users\neural_ninja\AppData\Local\Temp\ipykernel_10712\647575809.py:26: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return chain.run(supplier=supplier_name, context=context)


Processing SkyCloud Innovations 
Headquarters...
Processing CloudTech Solutions...
✅ Finished processing all vendors.

## CloudNexus Technologies 
Headquarters
```markdown
# CloudNexus Technologies Proposal Summary

## 1. Pricing
- **Standard Plan**: $20,000/month (covers up to 400 instances)
- **Enterprise Custom Plan**: Tailored pricing for large-scale deployments
- **Setup Fee**: $40,000 (one-time deployment and integration cost)
- **Additional Instances**: $45 per unit beyond plan limits
- **Discounts**: Available for multi-year agreements

## 2. Contract Terms
- **Minimum Subscription**: 12 months
- **Custom Solutions**: Available for large enterprises
- **Termination Policy**: 60-day notice required for contract cancellation

## 3. Compliance
- **Compliance Standards**: Adheres to SOC 2, NIST, HIPAA, and GDPR
- **Real-time Compliance Tracking**: Automated compliance documentation for audits
- **Security Assessments**: Bi-annual security assessments

## 4. Technical Capabilities
-